In [ ]:
import pandas as pd
from google.colab import drive
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
import statistics
from collections import Counter
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


drive.mount('/content/drive')

caminho_arquivo = '/content/drive/My Drive/agaricus-lepiota.data'

df = pd.read_csv(caminho_arquivo, header=None)
# one-hot encoding! Dummizar.
df_dummizado = pd.get_dummies(df, drop_first=True)

print(df.columns)
print(df_dummizado.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22],
           dtype='int64')
   0_p  1_c  1_f  1_k  1_s  1_x  2_g  2_s  2_y  3_c  ...  21_n  21_s  21_v  \
0    1    0    0    0    0    1    0    1    0    0  ...     0     1     0   
1    0    0    0    0    0    1    0    1    0    0  ...     1     0     0   
2    0    0    0    0    0    0    0    1    0    0  ...     1     0     0   
3    1    0    0    0    0    1    0    0    1    0  ...     0     1     0   
4    0    0    0    0    0    1    0    1    0    0  ...     0     0     0   

   21_y  22_g  22_l  22_m  22_p  22_u  22_w  
0     0     0     0     0     0     1     0  
1     0     1     0     0     0     0     0  
2     0     0     0     1     0     0     0  
3     0     0     0     0     0     1     0  
4     0     1     0     0    

In [ ]:
def impureza_gini_folha(alvo):
  '''calculo de impureza de uma folha'''

  if (len(alvo) == 0):
    return 1

  return 1 - (alvo.count(0)/len(alvo))**2 - (alvo.count(1)/len(alvo))**2

print(impureza_gini_folha(df_dummizado["0_p"].to_list()))

0.49935405449893955


In [ ]:
def impureza_gini_coluna(df, coluna):
    '''Calcula a impureza Gini para uma divisão baseada em uma coluna específica.'''
    gini_true = impureza_gini_folha(df[df[coluna] == 1]['0_p'].to_list())
    gini_false = impureza_gini_folha(df[df[coluna] == 0]['0_p'].to_list())

    media_ponderada = (gini_true * df[df[coluna] == 1].shape[0] +
                       gini_false * df[df[coluna] == 0].shape[0]) / df.shape[0]

    return media_ponderada


In [ ]:
def melhor_caracteristica_para_divisao(df):
    menor_impureza = float('inf')
    melhor_caracteristica = None

    for coluna in df.columns:
        if coluna == '0_p':  # Ignora a coluna target
            continue

        # Impureza para a divisão baseada na presença da característica
        df_left = df[df[coluna] == 0]  # Característica ausente
        df_right = df[df[coluna] == 1]  # Característica presente

        impureza_left = impureza_gini_folha(df_left['0_p'].to_list())
        impureza_right = impureza_gini_folha(df_right['0_p'].to_list())

        # Calcula a impureza total ponderada para a divisão
        n = len(df)
        impureza_total = (len(df_left) / n) * impureza_left + (len(df_right) / n) * impureza_right

        # Atualizando a melhor característica se a impureza total atual for menor
        if impureza_total < menor_impureza:
            menor_impureza = impureza_total
            melhor_caracteristica = coluna

    return melhor_caracteristica


In [ ]:
# Dividir o dataset em conjuntos de treino e teste
X = df_dummizado.drop(alvo, axis=1)
y = df_dummizado[alvo]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

# Treinar a Random Forest
forest = random_forest(train_df, n_trees=10, max_depth=10)

# Testar o Modelo
predictions = predict_forest(forest, test_df.drop(alvo, axis=1))

In [ ]:
alvo = '0_p'

def bootstrap_sample(df):
    n_samples = df.shape[0]
    indices = np.random.choice(n_samples, size=n_samples, replace=True)
    return df.iloc[indices]

def build_tree(df, depth=0, max_depth=10):
  '''Arvore de decisão adaptada para Random Forest'''
    # Verificar condição de parada
  if len(df[alvo].unique()) == 1 or depth == max_depth:
        return df[alvo].mode()[0]

  melhor_caracteristica = melhor_caracteristica_para_divisao(df)
  left_df = df[df[melhor_caracteristica] == 0]
  right_df = df[df[melhor_caracteristica] == 1]
  # Recursivamente construir subárvores
  left_tree = build_tree(left_df, depth + 1, max_depth)
  right_tree = build_tree(right_df, depth + 1, max_depth)
  return {'caracteristica': melhor_caracteristica, 'left': left_tree, 'right': right_tree}


In [ ]:
def predict_tree(tree, row):
  '''Prever com uma árvore'''
  if not isinstance(tree, dict):  # Verifica se é um nó folha
      return tree
  if row[tree['caracteristica']] == 0:
      return predict_tree(tree['left'], row)
  else:
      return predict_tree(tree['right'], row)





In [ ]:
def random_forest(df, n_trees=10, max_depth=10):
    trees = []
    for _ in range(n_trees):
        df_sample = bootstrap_sample(df)
        tree = build_tree(df_sample, max_depth=max_depth)
        trees.append(tree)
    return trees

def predict_forest(forest, X_test):
  '''Prever com a Random Forest'''

  # Inicializar uma lista para armazenar as predições finais para cada observação
  final_predictions = []
  # Iterar sobre cada observação no conjunto de teste
  for index, row in X_test.iterrows():
      # Coletar as predições de todas as árvores para a observação atual
      preds = [predict_tree(tree, row) for tree in forest]
      # Realizar a votação majoritária para a observação atual
      most_common_pred = Counter(preds).most_common(1)[0][0]
      # Adicionar a predição majoritária à lista de predições finais
      final_predictions.append(most_common_pred)
  return final_predictions




In [ ]:
def classifica_forest(forest, entrada):
    predicoes = [classifica_arvore(arvore, entrada) for arvore in forest]
    resultado = Counter(predicoes).most_common(1)[0][0]
    return resultado



In [ ]:
def classifica_arvore(arvore, entrada):
    if not isinstance(arvore, dict):  # Verifica se é um nó folha
        return arvore
    caracteristica = arvore['caracteristica']
    if entrada[caracteristica] == 0:
        return classifica_arvore(arvore['left'], entrada)
    else:
        return classifica_arvore(arvore['right'], entrada)


In [ ]:

def matriz_confusao_random_forest(df, porcentagem):
    treino = df.sample(frac=porcentagem)
    teste = df.drop(treino.index)

    confusao = [[0, 0], [0, 0]]

    forest = random_forest(treino, n_trees=10, max_depth=10)

    for indice, linha in teste.iterrows():
        classificacao = classifica_forest(forest, linha.drop(alvo))
        real = linha[alvo]

        if classificacao == 1 and real == 1:
            confusao[0][0] += 1  # Verdadeiro positivo
        elif classificacao == 1 and real == 0:
            confusao[0][1] += 1  # Falso positivo
        elif classificacao == 0 and real == 1:
            confusao[1][0] += 1  # Falso negativo
        elif classificacao == 0 and real == 0:
            confusao[1][1] += 1  # Verdadeiro negativo

    return confusao

# Calcular a matriz de confusão usando a Random Forest
matriz_confusao_rf = matriz_confusao_random_forest(df_dummizado, 0.75)

# Calcular métricas a partir da matriz de confusão
sensitivity_rf = matriz_confusao_rf[0][0] / (matriz_confusao_rf[0][0] + matriz_confusao_rf[1][0])
specificity_rf = matriz_confusao_rf[1][1] / (matriz_confusao_rf[0][1] + matriz_confusao_rf[1][1])
precision_rf = matriz_confusao_rf[0][0] / (matriz_confusao_rf[0][0] + matriz_confusao_rf[0][1])

print(matriz_confusao_rf)
print("Sensitivity (Random Forest):", sensitivity_rf)
print("Specificity (Random Forest):", specificity_rf)
print("Precision (Random Forest):", precision_rf)


[[977, 0], [0, 1054]]
Sensitivity (Random Forest): 1.0
Specificity (Random Forest): 1.0
Precision (Random Forest): 1.0
